# Basics with string sequence

* Find this notebook at `EpyNN/nnlive/dummy_string/train.ipynb`.
* Regular python code at `EpyNN/nnlive/dummy_string/train.py`.

In this notebook we will review:

* Handling sequential string data.
* Training of Feed-Forward (FF) and recurrent networks (RNN, LSTM, GRU).
* Differences between decisions and probabilities and related functions.

**It is assumed that all *basics* notebooks were already reviewed:**

* [Basics with Perceptron (P)](../dummy_boolean/train.ipynb)

**<font color='red'>This notebook does not enhance, extend or replace EpyNN's documentation.</font>**

**Relevant documentation pages for the current notebook:**

* [Fully Connected (Dense)](https://epynn.net/Dense.html)
* [Recurrent Neural Network (RNN)](https://epynn.net/RNN.html)
* [Long Short-Term Memory (LSTM)](https://epynn.net/LSTM.html)
* [Gated Recurrent Unit (GRU)](https://epynn.net/GRU.html)

## Environment and data

Follow [this link](prepare_dataset.ipynb) for details about data preparation.

Briefly, these dummy string data consists of sequence of characters. Sample features are each represented by one sequence and can be either associated with a positive or negative label.

Positive sequences are met when the first element in the sequence is equal to the last element in this same sequence, and reciprocally.

In [1]:
# EpyNN/nnlive/dummy_string/train.ipynb
# Standard library imports
import random

# Related third party imports
import numpy as np

# Local application/library specific imports
import nnlibs.initialize
from nnlibs.commons.io import one_hot_decode_sequence
from nnlibs.commons.maths import relu, softmax
from nnlibs.commons.library import (
    configure_directory,
    read_model,
)
from nnlibs.network.models import EpyNN
from nnlibs.embedding.models import Embedding
from nnlibs.flatten.models import Flatten
from nnlibs.rnn.models import RNN
from nnlibs.gru.models import GRU
from nnlibs.lstm.models import LSTM
from nnlibs.dense.models import Dense
from prepare_dataset import prepare_dataset
from settings import se_hPars


########################## CONFIGURE ##########################
random.seed(1)

np.set_printoptions(threshold=10)

np.seterr(all='warn')

configure_directory()


############################ DATASET ##########################
X_features, Y_label = prepare_dataset(N_SAMPLES=480)

Make: /media/synthase/beta/EpyNN/nnlive/dummy_string/datasets
Make: /media/synthase/beta/EpyNN/nnlive/dummy_string/models
Make: /media/synthase/beta/EpyNN/nnlive/dummy_string/plots


Let's control what we retrieved.

In [2]:
print(len(X_features))
print(len(X_features[0]))
print(X_features[0])
print(Y_label[0])

480
12
['G', 'A', 'C', 'T', 'T', 'G', 'G', 'C', 'C', 'A', 'T', 'C']
0


We retrieved a set sample features describing ``480`` samples.

Each sample is described by ``12`` string features.

Herein the label is ``0`` because the first and last element are different.

## Feed-Forward (FF)

To compare Feed-Forward and recurrent networks, we are goign to train a simple Perceptron first.

### Embedding

The principle of [One-hot encoding of string features](prepare_dataset.ipynb#One-hot-encoding-of-string-features) was detailed before.

Briefly, we can not do math on string data. Therefore, the one-hot encoding process may be summarized as such:

* List of all elements of size vocab_size. This basically answers: what is the number of distinct elements we can find in your data?
* Each element is associated with one index in the range(0, vocab_size). This provides a element_to_idx encoder.
* For each element in one sample features, a zero array is initialized. This array is set to one at the index which is assigned to the element in the encoder.

This is achieved during instantiation of the *embedding* layer by setting up ``X_encode=True``.

In [3]:
embedding = Embedding(X_data=X_features,
                      Y_data=Y_label,
                      X_encode=True,
                      Y_encode=True,
                      relative_size=(2, 1, 0))

Let's inspect some properties.

In [4]:
print(embedding.e2i)  # element_to_idx
print(embedding.dtrain.X[0])

print(embedding.i2e)  # idx_to_element
print(one_hot_decode_sequence(embedding.dtrain.X[0], embedding.i2e))

{'A': 0, 'C': 1, 'G': 2, 'T': 3}
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
{0: 'A', 1: 'C', 2: 'G', 3: 'T'}
['G', 'A', 'C', 'T', 'T', 'G', 'G', 'C', 'C', 'A', 'T', 'C']


Encoded sequences may be decoded as shown above.

### Flatten-Dense - Perceptron

Let's inspect the shape of the data.

In [5]:
print(embedding.dtrain.X.shape)

(320, 12, 4)


It contains 320 samples (m), each described by a sequence of 12 features (s) containing 4 elements (v).

12 features is the length of the sequences and 4 elements is the size of the vocabulary. Remember one-hot encoding makes a zero array of this size and sets 1 at the index corresponding to the element being encoded.

Still, the fully-connected or *dense* layer can only process bi-dimensionnal input arrays. That is the reason why we need to invoke a *flatten* layer in between the *embedding* and *dense* layer.

In [6]:
name = 'Flatten_Dense-2-softmax'

se_hPars['learning_rate'] = 0.001

flatten = Flatten()

dense = Dense(2, softmax)

layers = [embedding, flatten, dense]

model = EpyNN(layers=layers, name=name)

Initialize using most classically a *MSE* or *Binary Cross Entropy* loss function.

In [7]:
model.initialize(loss='BCE', seed=1, se_hPars=se_hPars.copy(), end='\r')

Train for hundred epochs.

In [8]:
model.train(epochs=50, init_logs=False)

Epoch 49 - Batch 0/0 - Accuracy: 0.738 Cost: 0.55307 - TIME: 2.98s RATE: 6.82e+03e/s TTC: 0s        

+-------+----------+-------+----------+-------+-------+------------------------------------+
| epoch |  lrate   |       | accuracy |       |  BCE  |             Experiment             |
|       |  Dense   |  (0)  |   (1)    |  (0)  |  (1)  |                                    |
+-------+----------+-------+----------+-------+-------+------------------------------------+
|   0   | 1.00e-03 | 0.556 |  0.556   | 0.729 | 0.710 | 1631698458_Flatten_Dense-2-softmax |
|   5   | 1.00e-03 | 0.712 |  0.738   | 0.620 | 0.573 | 1631698458_Flatten_Dense-2-softmax |
|  10   | 1.00e-03 | 0.728 |  0.744   | 0.600 | 0.567 | 1631698458_Flatten_Dense-2-softmax |
|  15   | 1.00e-03 | 0.728 |  0.744   | 0.586 | 0.565 | 1631698458_Flatten_Dense-2-softmax |
|  20   | 1.00e-03 | 0.731 |  0.750   | 0.575 | 0.565 | 1631698458_Flatten_Dense-2-softmax |
|  25   | 1.00e-03 | 0.731 |  0.756   | 0.568 | 0.565 | 16316

Plot the results.

In [9]:
model.plot(path=False)

Strictly talking, the Perceptron seems to have converged in the good direction.

By reducing the learning rate, all other things being equal, we obtained greater accuracy, lower cost and smoother curves on the plot.

You may have observed something possibly counter-intuituve: the cost, which describes the mean difference between experimental and expected outputs, is different between the training and testing set whereas the accuracy is the same.

That's because accuracy compares **decisions** and labels, whereas the cost from the loss function compare **probabilities** and labels.

For code, maths and pictures behind the *Flatten* and *Dense* layers, follow these link:

* [Flatten - Adapter](https://epynn.net/Flatten.html)
* [Fully Connected (Dense)](https://epynn.net/Dense.html)

Let's take a break and understand the difference in addition to make clear some semantics.

### Difference between accuracy and cost

The question is: Can we expect identical costs for the training and testing set **if** the accuracy is identical for the training and testing set?

Let's compare what outputs the *dense* layer (A) to the set of sample label (Y).

In [10]:
# This is probability distributions for each sample. 
print(model.embedding.dtrain.A)
print(model.embedding.dtrain.A.shape)

# These are the labels we target
print(model.embedding.dtrain.Y) 
print(model.embedding.dtrain.Y.shape) 

[[0.66591406 0.33408594]
 [0.86094668 0.13905332]
 [0.77311108 0.22688892]
 ...
 [0.89296296 0.10703704]
 [0.81178023 0.18821977]
 [0.74004683 0.25995317]]
(320, 2)
[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]
(320, 2)


We have probabilities (A) versus binary values (Y).

To compute the accuracy, one need to convert probabilities to decision, as well as to retrieve single-digit labels.

In [11]:
print(np.argmax(model.embedding.dtrain.A, axis=1))

# Equivalent to calling model.embedding.dtrain.y directly
print(np.argmax(model.embedding.dtrain.Y, axis=1))

[0 0 0 ... 0 0 0]
[0 0 1 ... 1 0 0]


Then, accuracy is computed such as:

In [12]:
print ((np.argmax(model.embedding.dtrain.A, axis=1) == model.embedding.dtrain.y))
print ((np.argmax(model.embedding.dtrain.A, axis=1) == model.embedding.dtrain.y).mean())

[ True  True False ... False  True  True]
0.7375


The cost is computed from **probabilities, not from decisions**. This apart from the fact that accuracy and cost are simply two different functions.

To compute a cost, we first need to compute the loss, which provides information for each single probability in the array (A).

In [13]:
# This is the cost. It deals with "true" labels against probabilities

loss = model.training_loss(model.embedding.dtrain.Y, model.embedding.dtrain.A)

print(loss.shape)

(320,)


The cost is a form of average of the loss. Whereas the loss in an array from element-wise comparison between probabilities and label, the cost is a scalar which is an average per sample, itself an average of the element-wise loss for this sample.

In [14]:
print(loss.shape)          # Averaged for each sample
print(loss.mean().shape)   # Average of above - scalar

cost = loss.mean()

print(cost)

(320,)
()
0.5527806811447783


Note that what is fed back in the network during the backward propagation phase is not the loss. It is the **derivative** of the loss.

In [15]:
dloss = model.training_loss(model.embedding.dtrain.Y, model.embedding.dtrain.A, deriv=True)

print(loss)
print(dloss)    # dloss is referred to as dA

[0.40659465 0.1497227  1.48329471 ... 2.23458032 0.20852563 0.30104181]
[[-0.75084764  0.75084764]
 [-0.58075606  0.58075606]
 [ 2.20372151 -2.20372151]
 ...
 [ 4.67128005 -4.67128005]
 [-0.61593025  0.61593025]
 [-0.67563292  0.67563292]]


The loss function and derivatives natively provided with EpyNN can be found in `EpyNN/nnlibs/commons/loss.py`.

The metrics natively provided with EpyNN can be found in `EpyNN/nnlibs/commons/metrics.py`.

## Recurrent Architectures

Herein, we are going to chain simple schemes based on recurrent architectures.

There are three most commonly cited recurrent layers:

* **Recurrent Neural Network (RNN)**: This is the most simple recurrent layer. It is composed of one to many recurrent units. Each cell performs a single activation which output the *hidden cell state* or simply *hidden state*.
* **Long Short-Term Memory (LSTM)**: By contrast with the RNN cell, the LSTM cell requires four activation which corresponds to three different gates: forget, input (two activations), and output. To compute the hidden cell state, it then requires a fifth activation. Note that in addition to the hidden cell state, there is another so-called cell *memory* state.
* **Gated Recurrent Unit (GRU)**: Compared to the LSTM cell, the GRU cell has only two gates: reset and update. Practically talking, GRU trains faster than LSTM and is reported to perform better on small dataset or shorter sequences. Both GRU and LSTM, however, are state-of-the art architecture to deal with sequential data.

See [here](nnlive/dummy_time/train.html#Recurrent-Neural-Network-(RNN)) for more detailed practical description or simply via the pages linked on top of this notebook.

### Embedding

In this example, we use the same setup than for Feed-Forward networks.

In [16]:
embedding = Embedding(X_data=X_features,
                      Y_data=Y_label,
                      X_encode=True,
                      Y_encode=True,
                      relative_size=(2, 1, 0))

We can now chain the simplest schemes to train binary classifiers based on recurrent layers.

### RNN-Dense

The number of RNN units in the RNN layer is set to 1.

In [17]:
name = 'RNN-1_Dense-2-softmax'

se_hPars['learning_rate'] = 0.001

rnn = RNN(1)

dense = Dense(2, softmax)

layers = [embedding, rnn, dense]

model = EpyNN(layers=layers, name=name)

Initialize the model.

In [18]:
model.initialize(loss='BCE', seed=1, se_hPars=se_hPars.copy(), end='\r')

Train for 50 epochs.

In [19]:
model.train(epochs=10, init_logs=False)

Epoch 9 - Batch 0/0 - Accuracy: 0.734 Cost: 0.58778 - TIME: 2.61s RATE: 1.16e+03e/s TTC: 0s         

+-------+----------+----------+-------+----------+-------+-------+----------------------------------+
| epoch |  lrate   |  lrate   |       | accuracy |       |  BCE  |            Experiment            |
|       |   RNN    |  Dense   |  (0)  |   (1)    |  (0)  |  (1)  |                                  |
+-------+----------+----------+-------+----------+-------+-------+----------------------------------+
|   0   | 1.00e-03 | 1.00e-03 | 0.584 |  0.500   | 0.659 | 0.671 | 1631698463_RNN-1_Dense-2-softmax |
|   1   | 1.00e-03 | 1.00e-03 | 0.656 |  0.612   | 0.638 | 0.644 | 1631698463_RNN-1_Dense-2-softmax |
|   2   | 1.00e-03 | 1.00e-03 | 0.734 |  0.762   | 0.623 | 0.624 | 1631698463_RNN-1_Dense-2-softmax |
|   3   | 1.00e-03 | 1.00e-03 | 0.734 |  0.762   | 0.612 | 0.610 | 1631698463_RNN-1_Dense-2-softmax |
|   4   | 1.00e-03 | 1.00e-03 | 0.734 |  0.762   | 0.604 | 0.598 | 1631698463_RNN-

You may already note that results are virtually identical to just using a basic Perceptron, although slightly better.

In [20]:
model.plot(path=False)

While the y-scale on the plot is a bit missleading when looking at the accuracy, there is no overfitting in there because the BCE cost is the same for both training and testing set at the end of the regression.

For code, maths and pictures behind the *RNN* layer, follow this link:

* [Recurrent Neural Network (RNN)](https://epynn.net/RNN.html)

### LSTM-Dense

Let's now proceed with an *LSTM* layer composed of the 1 unit, all other things being equal.

In [21]:
name = 'LSTM-1_Dense-2-softmax'

se_hPars['learning_rate'] = 0.005

lstm = LSTM(1)

dense = Dense(2, softmax)

layers = [embedding, lstm, dense]

model = EpyNN(layers=layers, name=name)

Initialize the model.

In [22]:
model.initialize(loss='BCE', seed=1, se_hPars=se_hPars.copy(), end='\r')

Train for 50 epochs.

In [23]:
model.train(epochs=10, init_logs=False)

Epoch 9 - Batch 0/0 - Accuracy: 0.734 Cost: 0.57896 - TIME: 1.62s RATE: 5.54e+02e/s TTC: 0s         

+-------+----------+----------+-------+----------+-------+-------+-----------------------------------+
| epoch |  lrate   |  lrate   |       | accuracy |       |  BCE  |            Experiment             |
|       |   LSTM   |  Dense   |  (0)  |   (1)    |  (0)  |  (1)  |                                   |
+-------+----------+----------+-------+----------+-------+-------+-----------------------------------+
|   0   | 5.00e-03 | 5.00e-03 | 0.734 |  0.762   | 0.585 | 0.566 | 1631698465_LSTM-1_Dense-2-softmax |
|   1   | 5.00e-03 | 5.00e-03 | 0.734 |  0.762   | 0.580 | 0.556 | 1631698465_LSTM-1_Dense-2-softmax |
|   2   | 5.00e-03 | 5.00e-03 | 0.734 |  0.762   | 0.579 | 0.554 | 1631698465_LSTM-1_Dense-2-softmax |
|   3   | 5.00e-03 | 5.00e-03 | 0.734 |  0.762   | 0.579 | 0.553 | 1631698465_LSTM-1_Dense-2-softmax |
|   4   | 5.00e-03 | 5.00e-03 | 0.734 |  0.762   | 0.579 | 0.553 | 1631698

The accuracy metrics is simply identical to what we have seen with a simple *RNN*, which is much faster to compute. It is not significantly better than what we obtained from a simple Perceptron, itself way faster to compute than the *RNN* based network.

In [24]:
model.plot(path=False)

By contrast with the RNN-based network, we observe here a slight overfitting because the cost is lower for the training dataset compared to the testing dataset.

For code, maths and pictures behind the *LSTM* layer, follow this link:

* [Long Short-Term Memory (LSTM)](https://epynn.net/LSTM.html)

### GRU-Dense

Let's now proceed with a GRU layer, all other things being equal.

In [25]:
name = 'GRU-1_Dense-2-softmax'

se_hPars['learning_rate'] = 0.005

gru = GRU(1)

flatten = Flatten()

dense = Dense(2, softmax)

layers = [embedding, gru, dense]

model = EpyNN(layers=layers, name=name)

Initialize the network.

In [26]:
model.initialize(loss='BCE', seed=1, se_hPars=se_hPars.copy(), end='\r')

Train for 50 epochs.

In [27]:
model.train(epochs=10, init_logs=False)

Epoch 9 - Batch 0/0 - Accuracy: 0.734 Cost: 0.57968 - TIME: 0.61s RATE: 7.34e+02e/s TTC: 0s         

+-------+----------+----------+-------+----------+-------+-------+----------------------------------+
| epoch |  lrate   |  lrate   |       | accuracy |       |  BCE  |            Experiment            |
|       |   GRU    |  Dense   |  (0)  |   (1)    |  (0)  |  (1)  |                                  |
+-------+----------+----------+-------+----------+-------+-------+----------------------------------+
|   0   | 5.00e-03 | 5.00e-03 | 0.734 |  0.762   | 0.582 | 0.562 | 1631698467_GRU-1_Dense-2-softmax |
|   1   | 5.00e-03 | 5.00e-03 | 0.734 |  0.762   | 0.580 | 0.557 | 1631698467_GRU-1_Dense-2-softmax |
|   2   | 5.00e-03 | 5.00e-03 | 0.734 |  0.762   | 0.580 | 0.556 | 1631698467_GRU-1_Dense-2-softmax |
|   3   | 5.00e-03 | 5.00e-03 | 0.734 |  0.762   | 0.580 | 0.556 | 1631698467_GRU-1_Dense-2-softmax |
|   4   | 5.00e-03 | 5.00e-03 | 0.734 |  0.762   | 0.580 | 0.555 | 1631698467_GRU-

Plot the results.

In [28]:
model.plot(path=False)

Overall, and using this dummy dataset made of string features, there is no significant metrcis/cost difference from the simple Perceptron to recurrent RNN, GRU and LSTM. In this situation, one would favor the simple Perceptron because it computes faster. At least, it is important to note that the best architecture is not the fanciest, but simply the one that suits your needs and resources.

For code, maths and pictures behind the *GRU* layer, follow this link:

* [Gated Recurrent Unit (GRU)](https://epynn.net/GRU.html)

## Write, read & Predict

A trained model can be written on disk such as:

In [29]:
model.write()

# model.write(path=/your/custom/path)

Make: /media/synthase/beta/EpyNN/nnlive/dummy_string/models/1631698467_GRU-1_Dense-2-softmax.pickle


A model can be read from disk such as:

In [30]:
model = read_model()

# model = read_model(path=/your/custom/path)

We can retrieve new features and predict on them.

In [31]:
X_features, _ = prepare_dataset(N_SAMPLES=10)

dset = model.predict(X_features, X_encode=True)

Results can be extracted such as:

In [32]:
for n, pred, probs in zip(dset.ids, dset.P, dset.A):
    print(n, pred, probs)

0 0 [0.75278242 0.24721758]
1 0 [0.73513511 0.26486489]
2 0 [0.73912875 0.26087125]
3 0 [0.7353358 0.2646642]
4 0 [0.73412882 0.26587118]
5 0 [0.74691258 0.25308742]
6 0 [0.73011278 0.26988722]
7 0 [0.73147555 0.26852445]
8 0 [0.73852927 0.26147073]
9 0 [0.73678448 0.26321552]
